In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
!pip install textblob
from sklearn.model_selection import GridSearchCV

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
import regex as re
from nltk.stem import WordNetLemmatizer
import string

In [4]:
train=pd.read_csv("C:/Users/Ajay K Srivastava/train.csv")
val=pd.read_csv("C:/Users/Ajay K Srivastava/validation.csv")
test=pd.read_csv("C:/Users/Ajay K Srivastava/test.csv")

# Multinomial Naive Bayes Model (tfidf tokenizer) 

In [5]:
vectorizer = CountVectorizer()
vectorizer.fit(train.X_train)

CountVectorizer()

# Tf-Idf Tokenizer

In [6]:
bagofwords=vectorizer.vocabulary_
X_train=vectorizer.transform(train.X_train)
X_val=vectorizer.transform(val.X_val)
X_test=vectorizer.transform(test.X_test)
print(len(bagofwords))
X_train.shape,X_val.shape

7331


((4514, 7331), (502, 7331))

In [7]:
tfidf_transformer = TfidfTransformer().fit(X_train)
X_train_tfidf=tfidf_transformer.transform(X_train)
X_val_tfidf=tfidf_transformer.transform(X_val)
X_test_tfidf=tfidf_transformer.transform(X_test)
X_train_tfidf.shape,X_val_tfidf.shape

((4514, 7331), (502, 7331))

# Model Fitting and Hyperparameter Tuning

In [8]:
for i in np.arange(0.25, 2.25, 0.25):
    spam_detector = MultinomialNB(alpha=i).fit(X_train_tfidf, train.y_train)
    y_pred = spam_detector.predict(X_val_tfidf)
    print(accuracy_score(val.y_val, y_pred), '\n', classification_report(val.y_val, y_pred))

0.9900398406374502 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       1.00      0.92      0.96        62

    accuracy                           0.99       502
   macro avg       0.99      0.96      0.98       502
weighted avg       0.99      0.99      0.99       502

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       1.00      0.90      0.95        62

    accuracy                           0.99       502
   macro avg       0.99      0.95      0.97       502
weighted avg       0.99      0.99      0.99       502

0.9820717131474104 
               precision    recall  f1-score   support

         ham       0.98      1.00      0.99       440
        spam       1.00      0.85      0.92        62

    accuracy                           0.98       502
   macro avg       0.99      0.93      0.96       502
weighted av

# Alpha = 0.5 gives best results. It maximises accuracy and recall for Spam mails.

In [9]:
spam_detector = MultinomialNB(alpha=0.5).fit(X_train_tfidf, train.y_train)

In [10]:
y_pred = spam_detector.predict(X_val_tfidf)
accuracy_score(val.y_val, y_pred)

0.9880478087649402

In [11]:
print(classification_report(val.y_val, y_pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       1.00      0.90      0.95        62

    accuracy                           0.99       502
   macro avg       0.99      0.95      0.97       502
weighted avg       0.99      0.99      0.99       502



In [12]:
y_pred_test = spam_detector.predict(X_test_tfidf)
accuracy_score(test.y_test, y_pred_test)

0.982078853046595

# Model Evaluation on Test Data

In [13]:
print(classification_report(test.y_test, y_pred_test))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       478
        spam       1.00      0.88      0.93        80

    accuracy                           0.98       558
   macro avg       0.99      0.94      0.96       558
weighted avg       0.98      0.98      0.98       558



# Multinomial Naive Bayes Model (Count tokenizer)

In [14]:
vectorizer = CountVectorizer()
vectorizer.fit(train.X_train)

CountVectorizer()

In [15]:
bagofwords=vectorizer.vocabulary_
X_train=vectorizer.transform(train.X_train)
X_val=vectorizer.transform(val.X_val)
X_test=vectorizer.transform(test.X_test)
print(len(bagofwords))
X_train.shape,X_val.shape

7331


((4514, 7331), (502, 7331))

In [16]:
for i in np.arange(0.25, 2.25, 0.25):
    spam_detector = MultinomialNB(alpha=i).fit(X_train, train.y_train)
    y_pred = spam_detector.predict(X_val)
    print(accuracy_score(val.y_val, y_pred), '\n', classification_report(val.y_val, y_pred))


0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted avg       0.99      0.99      0.99       502

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted avg       0.99      0.99      0.99       502

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted av

# The model performs similar for all values of alpha from 0.5 to 2

# Support Vector Classifier Model

In [17]:
clf = SVC(C=1)
clf.fit(X_train_tfidf, train.y_train)


SVC(C=1)

In [18]:
y_predicted = clf.predict(X_val_tfidf)
print(accuracy_score(val.y_val, y_pred), '\n', classification_report(val.y_val, y_pred))

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted avg       0.99      0.99      0.99       502



In [19]:
for i in range(1,10000,1000):
    clf = SVC(C=i)
    clf.fit(X_train_tfidf, train.y_train)
    y_predicted = clf.predict(X_val_tfidf)
    print(accuracy_score(val.y_val, y_pred), '\n', classification_report(val.y_val, y_pred))

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted avg       0.99      0.99      0.99       502

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted avg       0.99      0.99      0.99       502

0.9880478087649402 
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99       440
        spam       0.97      0.94      0.95        62

    accuracy                           0.99       502
   macro avg       0.98      0.97      0.97       502
weighted av

# Model performs similar for the different values of regularization parameter.